# Nivell 1 / Nivell 2

## Exercici 1 / Exercici 2

#### Normalitza, identifica i enumera cada un dels atributs / variables de l'estructura de l'arxiu "Web_access_log-akumenius.com" que trobaràs al repositori de GitHub "Data-sources".

#### Neteja, preprocesa, estructura i transforma (dataframe) les dades del registre d'Accés a la web.

In [101]:
import pandas as pd
import numpy as np
import re

In [55]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 1000)

In [159]:
#Read file
df_webpage_logs = pd.read_csv('C:/Users/xizes/Web_access_log-akumenius.com.txt',header=None, sep='"', encoding='utf-8', engine='python', na_values=np.nan, names=range(9))

In [155]:
#Create regex poattern to separate each relevant information from first column
pattern_host = r'(\blocalhost\b|\bw?w?w?.\w+.com|w?w?w?.\w+.es\b)'
pattern_ip = r'127\.0\.0\.1|\b\d?\d?\d.\d?\d?\d.\d?\d?\d.\d?\d?\d\b'
pattern_date = r'\d\d\/\D\D\D\/\d{4}\:\d\d\:\d\d\:\d\d\s\+\d\d\d\d'

In [163]:
# Create columns to receive the separated values 
df_webpage_logs['host'] = ''
df_webpage_logs['ip'] = ''
df_webpage_logs['date'] = ''

In [164]:
#Iterate through the first column and extract each information accordingly to the regex patterns
for i, line in enumerate(df_webpage_logs[0]):    
    df_webpage_logs['host'].iloc[i] = re.findall(pattern=pattern_host, string=line)[0]
    df_webpage_logs['ip'].iloc[i] = re.findall(pattern=pattern_ip, string=line)[0]
    df_webpage_logs['date'].iloc[i] = re.findall(pattern=pattern_date, string=line)[0]

In [169]:
#Support Cell - Ignore
#print(re.findall(pattern=pattern_host, string=df_webpage_logs[0].iloc[147]))
#print(re.findall(pattern=pattern_ip, string=df_webpage_logs[0].iloc[14785]))
#print(re.findall(pattern=pattern_date, string=df_webpage_logs[0].iloc[14785]))

In [511]:
#Support cell - Ignore
#df_webpage_logs[0].iloc[261850]

In [523]:
#df_webpage_logs.tail(750)

In [434]:
#Bk Up
#df_webpage_logs_2.to_csv('C:/Users/xizes/web_logs.csv')

In [180]:
#Change columns names
df_webpage_logs.rename(columns={1:'request', 2:'response', 3:'web', 5:'browser'}, inplace=True)

In [238]:
#Support cell - Ignore
#df_webpage_logs[df_webpage_logs[6] != ' VLOG=-']

In [229]:
#Drop irrelevant column
df_webpage_logs.drop(columns=[6], inplace=True)

In [233]:
#Change columns order
cols = df_webpage_logs.columns.to_list()

cols_2 = ['host', 'ip', 'date', 'request', 'response', 'browser', 'web']

df_webpage_logs = df_webpage_logs[cols_2]

In [240]:
#df_webpage_logs.head(750)

## Exercici 3

#### Geolocalitza les IP's.

In [525]:
#df_webpage_logs.head(30)

In [248]:
#Create unique ip´s list
lis_ips = list(df_webpage_logs['ip'].unique())

In [250]:
len(lis_ips)

2921

In [527]:
#lis_ips

**Get ip location using the module ip2geotools**

In [282]:
#! pip install ip2geotools

In [312]:
from ip2geotools.databases.noncommercial import DbIpCity
import json
from time import sleep

In [311]:
DbIpCity.get('162.243.192.191', api_key='free').to_json()

'{"ip_address": "162.243.192.191", "city": "New York", "region": "New York", "country": "US", "latitude": 40.7127281, "longitude": -74.0060152}'

In [266]:
#Create dict containing each unique ip and their own dicts to gather all information possible
dic_ips = {}

for e in lis_ips:
    dic_ips[e] = {}
    

In [313]:
for ip in dic_ips.keys():
    response = json.loads(DbIpCity.get(ip, api_key='free').to_json())    
    dic_ips[ip] = response
    sleep(2)

InvalidRequestError: 

In [529]:
#dic_ips

After trying, twice, to use **ip2geotools** - the 1st time alone, and the 2nd time using sleep - I could get more results using sleep, but after a while I got the same error: 'InvalidRequestError'. So I´ll try **ipInfo** API.

In [387]:
#pip install ipinfo

In [388]:
import ipinfo

In [397]:
#Setting basic variables + testing
access_token = 'badbd60180de49'
handler = ipinfo.getHandler(access_token)
ip_address = '216.239.36.21'
response = handler.getDetails(ip_address)

In [398]:
response.all

{'ip': '216.239.36.21',
 'hostname': 'any-in-2415.1e100.net',
 'anycast': True,
 'city': 'Mountain View',
 'region': 'California',
 'country': 'US',
 'loc': '37.4056,-122.0775',
 'org': 'AS15169 Google LLC',
 'postal': '94043',
 'timezone': 'America/Los_Angeles',
 'country_name': 'United States',
 'latitude': '37.4056',
 'longitude': '-122.0775'}

In [403]:
#Iterate through the ip dict to collect all the info of each ip
for ip in dic_ips.keys():
    response = handler.getDetails(ip)
    dic_ips[ip] = response.all

In [424]:
#Exclude all hosts == 'localhost'
df_webpage_logs_2 = df_webpage_logs[df_webpage_logs['host'] != 'localhost']

In [431]:
#Reset index
df_webpage_logs_2.reset_index(drop=True, inplace=True)

In [484]:
#dic_ips

In [442]:
df_ips_loc = pd.DataFrame(data=dic_ips.values())

# Nivell 3

## Exercici 3

#### Mostra'm la teva creativitat, Sorprèn-me fes un pas més enllà amb l'anàlisi anterior.

In [543]:
import plotly.express as px
from plotly.offline import plot, iplot, init_notebook_mode
#import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [531]:
#df_ips_loc.head()

In [445]:
#Iniciate column to receive ip counts
df_ips_loc['count'] = ''

In [454]:
#Count unique ip´s
ips_count = df_webpage_logs_2.groupby('ip')['host'].count()

In [475]:
#Compare the df and ips_count to aggregate the counts into the df
for e, ip in enumerate(df_ips_loc['ip']):
    for index in ips_count.index:     
        if ip == index:
            #print(e, ip, index, ips_count[index])
            df_ips_loc['count'].iloc[e] = ips_count[index]
        else:
            continue

In [496]:
#Group the ip´s per country
f = df_ips_loc.groupby('country_name')['ip'].count()

f = pd.DataFrame(data=f) 

In [547]:
#Plot the ip count per country
plot_map_ip = px.choropleth(data_frame=f, locations=f.index, locationmode='country names', color='ip', title='IP connectinons per Country')
plot_map_ip.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})
plot_map_ip.show()